In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv('./processed_df.csv')
display(df.head())
df.info()

,연령,종교,직업,혼인상태,교육수준,건강상태,전체 만족도,소득수준,활동성,target
0,67,1,3.0,1,3,0,1,2,1,1
1,68,2,2.0,3,4,0,1,2,1,1
2,66,3,1.0,1,2,0,2,3,0,1
3,68,3,3.0,3,4,0,1,2,1,3
4,73,3,6.0,3,4,0,1,2,0,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2092 entries, 0 to 2091
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   연령      2092 non-null   int64  
 1   종교      2092 non-null   int64  
 2   직업      2092 non-null   float64
 3   혼인상태    2092 non-null   int64  
 4   교육수준    2092 non-null   int64  
 5   건강상태    2092 non-null   int64  
 6   전체 만족도  2092 non-null   int64  
 7   소득수준    2092 non-null   int64  
 8   활동성     2092 non-null   int64  
 9   target  2092 non-null   int64  
dtypes: float64(1), int64(9)
memory usage: 163.6 KB


- 데이터셋 분리

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    df.iloc[:,:-1], df.iloc[:,-1], test_size=0.2, random_state=2022)

- 오버샘플링 데이터 생성

In [7]:
from imblearn.over_sampling import SMOTE
oversampling_instance = SMOTE()
X_train_o, y_train_o = oversampling_instance.fit_resample(X_train, y_train)
y_train_o.value_counts()

1    1024
2    1024
3    1024
4    1024
5    1024
Name: target, dtype: int64

- XGBoost

In [8]:
from xgboost import XGBClassifier
xgb = XGBClassifier(n_estimators=400, learning_rate=0.1, random_state=2022)
evals = [(X_test, y_test)]

In [9]:
xgb.fit(X_train, y_train, early_stopping_rounds = 100, eval_metric='merror', eval_set=evals , verbose=True)
xgb.score(X_test, y_test)

[0]	validation_0-merror:0.365155
Will train until validation_0-merror hasn't improved in 100 rounds.
[1]	validation_0-merror:0.362769
[2]	validation_0-merror:0.362769
[3]	validation_0-merror:0.362769
[4]	validation_0-merror:0.360382
[5]	validation_0-merror:0.365155
[6]	validation_0-merror:0.360382
[7]	validation_0-merror:0.365155
[8]	validation_0-merror:0.360382
[9]	validation_0-merror:0.360382
[10]	validation_0-merror:0.360382
[11]	validation_0-merror:0.360382
[12]	validation_0-merror:0.360382
[13]	validation_0-merror:0.360382
[14]	validation_0-merror:0.355609
[15]	validation_0-merror:0.355609
[16]	validation_0-merror:0.355609
[17]	validation_0-merror:0.355609
[18]	validation_0-merror:0.355609
[19]	validation_0-merror:0.355609
[20]	validation_0-merror:0.355609
[21]	validation_0-merror:0.355609
[22]	validation_0-merror:0.355609
[23]	validation_0-merror:0.355609
[24]	validation_0-merror:0.355609
[25]	validation_0-merror:0.355609
[26]	validation_0-merror:0.355609
[27]	validation_0-merror

0.6443914081145584

In [10]:
xgb.fit(X_train_o, y_train_o, early_stopping_rounds = 100, eval_metric='merror', eval_set=evals , verbose=True)
xgb.score(X_test, y_test)

[0]	validation_0-merror:0.553699
Will train until validation_0-merror hasn't improved in 100 rounds.
[1]	validation_0-merror:0.551313
[2]	validation_0-merror:0.546539
[3]	validation_0-merror:0.560859
[4]	validation_0-merror:0.560859
[5]	validation_0-merror:0.558473
[6]	validation_0-merror:0.548926
[7]	validation_0-merror:0.544153
[8]	validation_0-merror:0.551313
[9]	validation_0-merror:0.546539
[10]	validation_0-merror:0.551313
[11]	validation_0-merror:0.548926
[12]	validation_0-merror:0.541766
[13]	validation_0-merror:0.536993
[14]	validation_0-merror:0.539379
[15]	validation_0-merror:0.536993
[16]	validation_0-merror:0.539379
[17]	validation_0-merror:0.536993
[18]	validation_0-merror:0.539379
[19]	validation_0-merror:0.536993
[20]	validation_0-merror:0.534606
[21]	validation_0-merror:0.539379
[22]	validation_0-merror:0.539379
[23]	validation_0-merror:0.539379
[24]	validation_0-merror:0.536993
[25]	validation_0-merror:0.534606
[26]	validation_0-merror:0.529833
[27]	validation_0-merror

0.48448687350835323

- KNN

In [11]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)
knn.score(X_test, y_test)

0.6229116945107399

In [12]:
knn.fit(X_train_o, y_train_o)
knn.score(X_test, y_test)

0.3412887828162291

In [13]:
from sklearn.model_selection import GridSearchCV
params = {'n_neighbors': [5, 10, 15, 20]}
grid_knn = GridSearchCV(knn, params, scoring='accuracy', cv=5)
grid_knn.fit(X_train, y_train)
grid_knn.best_params_

{'n_neighbors': 20}

In [14]:
params = {'n_neighbors': [20, 25, 30]}
grid_knn = GridSearchCV(knn, params, scoring='accuracy', cv=5)
grid_knn.fit(X_train, y_train)
grid_knn.best_params_

{'n_neighbors': 30}

In [15]:
params = {'n_neighbors': [27, 28, 29]}
grid_knn = GridSearchCV(knn, params, scoring='accuracy', cv=5)
grid_knn.fit(X_train, y_train)
grid_knn.best_params_

{'n_neighbors': 28}

In [16]:
grid_knn.best_estimator_.score(X_test, y_test)

0.639618138424821

- CatBoost

In [19]:
# pip install catboost
from catboost import CatBoostClassifier
cb = CatBoostClassifier(iterations=100)

In [20]:
cb.fit(X_train, y_train)
cb.score(X_test, y_test)

Learning rate set to 0.5
0:	learn: 1.2295009	total: 52.7ms	remaining: 5.22s
1:	learn: 1.1677714	total: 63.8ms	remaining: 3.12s
2:	learn: 1.1409022	total: 66.5ms	remaining: 2.15s
3:	learn: 1.1146789	total: 79.4ms	remaining: 1.91s
4:	learn: 1.0922391	total: 82.9ms	remaining: 1.57s
5:	learn: 1.0714506	total: 89ms	remaining: 1.39s
6:	learn: 1.0563771	total: 93.8ms	remaining: 1.25s
7:	learn: 1.0363551	total: 96.9ms	remaining: 1.11s
8:	learn: 1.0275897	total: 99.3ms	remaining: 1s
9:	learn: 1.0207570	total: 101ms	remaining: 908ms
10:	learn: 1.0086050	total: 103ms	remaining: 830ms
11:	learn: 1.0044612	total: 104ms	remaining: 764ms
12:	learn: 1.0020809	total: 105ms	remaining: 705ms
13:	learn: 0.9935909	total: 107ms	remaining: 656ms
14:	learn: 0.9787064	total: 108ms	remaining: 613ms
15:	learn: 0.9753133	total: 110ms	remaining: 577ms
16:	learn: 0.9638325	total: 112ms	remaining: 548ms
17:	learn: 0.9573172	total: 114ms	remaining: 520ms
18:	learn: 0.9478027	total: 116ms	remaining: 493ms
19:	learn: 0

0.6109785202863962

In [21]:
cb.fit(X_train_o, y_train_o)
cb.score(X_test, y_test)

Learning rate set to 0.5
0:	learn: 1.4950878	total: 17.6ms	remaining: 1.75s
1:	learn: 1.4244008	total: 32ms	remaining: 1.57s
2:	learn: 1.3634333	total: 50.7ms	remaining: 1.64s
3:	learn: 1.3183534	total: 72.3ms	remaining: 1.74s
4:	learn: 1.2974217	total: 77.8ms	remaining: 1.48s
5:	learn: 1.2633202	total: 124ms	remaining: 1.94s
6:	learn: 1.2391835	total: 139ms	remaining: 1.85s
7:	learn: 1.2076223	total: 153ms	remaining: 1.76s
8:	learn: 1.1893189	total: 174ms	remaining: 1.76s
9:	learn: 1.1703097	total: 187ms	remaining: 1.68s
10:	learn: 1.1474355	total: 206ms	remaining: 1.67s
11:	learn: 1.1357901	total: 221ms	remaining: 1.62s
12:	learn: 1.1149522	total: 235ms	remaining: 1.57s
13:	learn: 1.0960249	total: 249ms	remaining: 1.53s
14:	learn: 1.0852466	total: 264ms	remaining: 1.5s
15:	learn: 1.0594466	total: 286ms	remaining: 1.5s
16:	learn: 1.0451090	total: 297ms	remaining: 1.45s
17:	learn: 1.0334318	total: 306ms	remaining: 1.39s
18:	learn: 1.0197910	total: 312ms	remaining: 1.33s
19:	learn: 1.00

0.441527446300716

- lightgbm

In [29]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=2022)
lgb.fit(X_train, y_train)
lgb.score(X_test, y_test)

0.6109785202863962

In [30]:
lgb.predict(X_test)

array([3, 1, 1, 1, 2, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 4, 1, 1, 1, 1, 1, 5, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1,
       5, 2, 1, 1, 1, 1, 4, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       4, 1, 3, 4, 1, 2, 1, 1, 1, 1, 1, 1, 1, 4, 5, 1, 1, 1, 1, 1, 3, 3,
       1, 3, 2, 1, 1, 1, 1, 5, 3, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 5,
       1, 5, 3, 1, 1, 4, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 5, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 5, 1, 1, 1, 3, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1,

In [31]:
lgb.fit(X_train_o, y_train_o)
lgb.score(X_test, y_test)

0.47494033412887826

In [32]:
params = {'num_leaves': (24, 26, 28),
          'max_depth': (5, 10, 15),}
grid_lgb = GridSearchCV(lgb, params, scoring='accuracy', cv=5)
grid_lgb.fit(X_train, y_train)
grid_lgb.best_params_

{'max_depth': 5, 'num_leaves': 24}

In [33]:
grid_lgb.best_estimator_.score(X_test, y_test)

0.6348448687350835

In [34]:
grid_lgb.predict(X_test)

array([1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1,
       5, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       4, 1, 5, 4, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 3, 1,
       1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 2, 1,
       1, 5, 3, 1, 5, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 5, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 4, 3, 1, 1, 1, 1, 1, 1, 1, 1,